<style>
pre > code {
    background-color: #3A3960 !important;
    padding: 10px;
    display: block;
    border-radius: 5px;
    border: 1px solid #ccc;
    overflow-x: auto;
}
</style>

# 4.4 Grundlegende FastAPI Konzepte: Database, Authentication, Authorization, Hashing Passwords

Nachdem wir viele Grundlagen von FastAPI anhand des Bücherbeispiels kennen gelernt haben, wollen wir weitere Konzepte an einer TO-DO App lernen. Die nächsten Schwerpunkte sind:
- Datenbanken: Wir arbeiten mit einer vollständigen SQL-Datenbank und lernen drei verschiedene Produktionsdatenbanken kennen:
  - SQLite (eingebettete Datenbank, einfacher Einstieg)
  - PostgreSQL & MySQL (leistungsfähige Datenbanken für echte Anwendungen)
- Benutzerauthentifizierung mit JWT:
  - Nutzer können sich mit Benutzernamen & Passwort registrieren und anmelden.
  - Passwörter werden gehasht, um die Sicherheit zu gewährleisten.
- Autorisierung & Rollenmanagement:
  - Benutzer erhalten unterschiedliche Rollen (z. B. Admin).
  - Admins haben Zugriff auf spezielle API-Endpunkte, die andere nicht nutzen können.
- Architektur & Sicherheit:
  - Unsere Webseite kommuniziert mit dem FastAPI-Server.
  - FastAPI verarbeitet Anfragen, überprüft Authentifizierung & Autorisierung und greift auf die Datenbank zu.
  - Wir implementieren moderne Sicherheitsmaßnahmen für eine professionelle API.

Als erstes erstellen wir eine Datei mit dem Namen "database.py". Anschließend installieren wir SQL-Alchemy:
```
pip install SQLAlchemy
```

SQLAlchemy ist eine Python-Bibliothek, die uns hilft, mit Datenbanken zu arbeiten.
Anstatt komplizierte SQL-Befehle zu schreiben, kann man SQLAlchemy für eine einfache und strukturierte Art der Datenbankkommunikation nutzen. Einige Vorteile sind:
- Man kann Datenbanktabellen als Python-Objekte behandeln.
- Es erleichtert die Verwaltung von Datenbanken (Erstellen, Ändern, Abfragen von Daten).
- Es funktioniert mit vielen Datenbanken (z. B. SQLite, MySQL, PostgreSQL).
- Es gibt dir zwei Möglichkeiten zu arbeiten: Mit reinem SQL oder mit ORM (Object-Relational Mapping).

ORM bedeutet, dass man Datenbanktabellen wie Python-Klassen behandelt.
Statt direkt SQL zu schreiben, nutzt man Python-Objekte, um Daten zu speichern, zu bearbeiten und abzurufen. Dank ORM hat man also folgende Vorteile:
- Man muss kein SQL schreiben, man arbeitet mit Python-Klassen und Objekten.
- Es macht den Code lesbarer und einfacher.
- Es ist sicherer, weil du dich nicht um SQL-Injections kümmern musst.
- Es funktioniert mit verschiedenen Datenbanktypen (du kannst leicht von SQLite zu PostgreSQL wechseln).

Als erstes erstellen wir eine Konstente:
```python
SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"
```

Diese URL wird verwendet, um einen Speicherort für die Datenbank anzugeben:
- "sqlite:///" gibt an, dass SQLite als Datenbank-Engine verwendet wird
- "./" bedeutet, dass die Datei im aktuellen Verzeichnis gespeichert wird.
- "todos.db" ist der Dateiname der SQLite-Datenbank.
- Allgemein "sqlite:///pfad/zur/datenbank.db"

Jetzt müssen wir von "sqlalchemy" die Funktion "create_engine" importieren:
- "create_engine" ist eine Funktion in SQLAlchemy, die eine Datenbank-Engine erstellt.
- Die Engine ist die Schnittstelle zwischen SQLAlchemy und der Datenbank.
- Sie verwaltet Verbindungen zur Datenbank und führt SQL-Befehle aus.

```python
from sqlalchemy import create_engine

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
```

Was genau macht eigentlich "connect_args={"check_same_thread": False}"?<br>
Dadurch erlaubt man mehreren Teilen des Programms, gleichzeitig mit der SQLite-Datenbank zu arbeiten. Wenn wir sagen "ein Teil des Programms", meinen wir eine Aufgabe oder eine Aktion, die das Programm gerade ausführt.
<br>
<br>
Warum ist diese Einstellung so wichtig?<br>
SQLite erlaubt normalerweise nur einem einzigen Teil des Programms, die Verbindung zur Datenbank zu benutzen. FastAPI arbeitet aber so, dass mehrere Teile des Programms gleichzeitig irgendwelche Dinge machen können. Ohne diese Einstellung würde SQLite einen Fehler ausgeben, weil es denkt: "Hey, jemand anderes benutzt meine Verbindung!" Diese Einstellung brauchst man nur für SQLite, nicht für größere Datenbanken wie PostgreSQL oder MySQL.
<br>
<br>
Stellt euch vor, man hat eine To-Do-App, bei der Benutzer Aufgaben speichern können.
Das Programm kann mehrere Dinge gleichzeitig tun, z. B.:
- Ein Benutzer speichert eine neue Aufgabe.
- Ein anderer Benutzer ruft seine gespeicherten Aufgaben ab.

Beide Aktionen passieren gleichzeitig, also gibt es zwei "Teile" des Programms, die auf die Datenbank zugreifen möchten. SQLite erlaubt standardmäßig nur einem einzigen Teil (z. B. nur dem Speichern oder nur dem Abrufen) die Datenbank zu benutzen.
Wenn FastAPI aber mehrere Dinge gleichzeitig macht, kann das zu einem Fehler führen.
<br>
<br>
Jetzt importieren wir noch sessionmaker. Es handelt sich um eine Funktion  in SQLAlchemy, mit der wir Datenbank-Sitzungen (Sessions) erstellen. Eine Session ist eine Art "Verbindung" zur Datenbank, mit der wir Daten abrufen, hinzufügen, aktualisieren oder löschen können.

```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
```

Jetzt erstellen wir eine Session Klasse, doe später genutzt wird, um eine Verbindung zur Datenbank herzustellen und Daten zu verwalten:

```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
```

- "autocommit=False": Legt fest, dass Änderungen nicht automatisch gespeichert werden. Wir müssen "db.commit()" aufrufen, um Änderungen dauerhaft zu machen
- "autoflush=False": Verhindert, dass Daten automatisch in die Datenbank geschrieben werden, bevor eine Abfrage erfolgt. Stattdessen müssen wir "db.flush()" oder "db.commit()" aufrufen, um Änderungen zu speicher
- "bind=engine": Bedeutet, dass diese Session mit unserer Datenbank-Engine (engine) verbunden ist. Dadurch weiß SQLAlchemy, mit welcher Datenbank die Sitzung arbeiten soll.

Jetzt müssen wir noch "declarative_base" importieren, um eine Basisklasse für unsere Datenbank-Modelle zu erstellen. Jede Tabelle, die wir in unserer Datenbank haben wollen, wird als Python-Klasse definiert und von Base abgeleitet:

```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()
```

Jetzt erstellen wir die Datei "models.py". Diese Datei wird verwendet um sogenannte Datenbank-Modelle (Tabellen) zu definieren. Das hilft, den Code übersichtlich zu halten, indem alle Datenbank-Strukturen an einem Ort organisiert sind. Wir werden eine Tabelle mit dem Namen "Todos" haben. In ihr sind Spalten wie z.B. "ID" oder "Titel" enthalten:

```python
from database import Base
from sqlalchemy import Column, Integer, String, Boolean

class Todos(Base):
    __tablename__ = "todos"
```

Dieser Code definiert also eine Tabelle namens "todo" für eine Datenbank mit SQLAlchemy ("class Todos(Base):"). Wie man sieht erben hier alle Klassen bzw. Tabellen von der Grundklasse "Base", welche wir in der "database.py" Datei erstellt hatten.
<br>
<br>
Außerdem haben wir noch vier Klassen importiert:
- "Column" wird dan nverwendet um Spalten in der Tabelle zu definieren.
- "Integer" wird dann verwendet, um zu zeigen, dass es sich um Ganzzahlen in einer Spalte handelt.
- "String" wird ebenfalls dann verwendet, um zu zeigen dass es sich in der Spalte um Einträge vom Datentyp String (Zeichenkette) handelt.
- - "Boolean" wird ebenfalls dann verwendet, um zu zeigen dass es sich in der Spalte um Einträge vom Datentyp Boolean (Wahrheitswerte) handelt.

Die Zeile `__tablename__ = "todos"` sagt dass die Tabelle in der Datenbank "todos" heißen soll.
<br>
<br>
Nun legen wir die einzelnen Spalten mit den dazugehörigen Datentypen an:

```python
from database import Base
from sqlalchemy import Column, Integer, String, Boolean

class Todos(Base):
    __tablename__ = "todos"
    
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String)
    description = Column(String)
    priority = Column(Integer)
    complete = Column(Boolean, default=False)
```

Die Zeile:
- "id = Column(Integer, primary_key=True, index=True)" erstellt eine Spalte mit dem Namen "id". Mit "index=True" sagen wir aus dass die Spalte schneller durchsucht werden kann. Man kann sich den "index", als eine Art Inhaltsverzeichnis eines Buchen sich vorstellen, damit können wir schneller Daten finden. Durch "primary_key=True", sagen wir dass die Spalte als Primärschlüssel behandelt werden soll. Jede Zeile in einer Tabelle braucht eine eindeutige Kennung (ID), um sie zu identifizieren. Diese ID darf sich nicht wiederholen und ist für jede Zeile einzigartig.
- "title = Column(String)" erstellt eine normale Spalte vom Datentyp String.
- "description = Column(String)" erstellt eine normale Spalte vom DAtentyp String
- "priority = Column(Integer)" erstellt eine normale Spalte vom Datentyp Integer.
- " complete = Column(Boolean, default=False)" erstellt eine normale Spalte vom Datentyp Boolean, wobei durch "default" wir von Anfang an einen Wert von "False" jeder Zelle zuweisen.

Jetzt ist es an der Zeit das Hauptprogramm bzw. den Einsteigspunkt unserer Anwendung zu erstellen. Dazu erzeugen wir die Datei "main.py". Wie wir es bereits kennen erstellen wir eine klassische FastAPI Anwendung mit der "FastAPI" Klasse. Außerdem importieren wir die Datei "models.py" und die "engine" aus database.py.
<br>
<br>
Die "engine" ist die Verbindung zur Datenbank. Durch "models.Base.metadata.create_all(bind=engine)" schaut sich SQLAlchemy alle Tabellen in der "models.py" Datei an. Falls die Tabellen noch nicht in der Datenbank existieren, werden sie erstellt.

**main.py:**
```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()
```
Jetzt starten wir den uvicorn Servern:
```
uvicorn main:app --reload
```

Dadurch wird die "main.py" Datei ausgeführt und auch natürlich die Datenbank Datei "todos.db", in dem selben Verzeichnis erzeugt:

<img src="../img/FastAPI_59.png" alt="FastAPI_01" width="400">

Damit wir angenehmer mit SQLite arbeiten können, installieren wir die Anwendung "command-line tools for managing SQLite database files". An dieser Stellen gehen wir von einem Windows Betriebssystem aus, für einen Mac ist es noch einfacher. Diese Anwendung bekommen wir von <br>
https://www.sqlite.org/download.html

<img src="../img/FastAPI_60.png" alt="FastAPI_01" width="400">

Anschließend entpacken wir den Ordner und fügen ihn unter `C:\` hinzu. Anscchließend bennen ich den Ordner von "sqlite-tools-win-x64-3490000" in "sqlite3. Anschließend können wir den Ordner "C:\sqlite3" zu den System-Umgebungsvariablen hinzufügen.
<br>
<br>
Als erstes geben wir in der Windowssuche unter Start "Edit the system enviroment variables". Dann klicken wir auf "Envidoment Variables":

<img src="../img/FastAPI_61.png" alt="FastAPI_01" width="400">

Unter "system variables" werden wir den Eintrag "Path" bearbeiten:

<img src="../img/FastAPI_62.png" alt="FastAPI_01" width="400">

Wir klicken auf "Path" mit Doppelklick und erzeugen durch "New" einen neuen Eintrag mit dem Pfad "C:\sqlite3" zu der sqlite3.exe Datei:

<img src="../img/FastAPI_63.png" alt="FastAPI_01" width="400">

Anschließend schließen wir alle Fenster durch "Ok". In der CMD können wir nun testen ob alles erfolgreich eingerichtet wurde. Dazu öffnen wir CMD und führen den folgenden Befehl aus:
```
sqlite3
```

Anschließen sollte so eine ähnliche Ausgabe erscheinen:

<img src="../img/FastAPI_64.png" alt="FastAPI_01" width="400">

Jetzt können wir uns mit grundlegenden SQL-Queries beschäftigen. Wir wollen Daten in unsere Datenbankdatei "todos.db" hinzufügen.
In unserem Terminal von VS Code, führen wir unter dem Projektverzeichnis folgenden Befehl aus:
```
sqlite3 todos.db
```

Anschließend soltle es etwa so aussehen:

<img src="../img/FastAPI_65.png" alt="FastAPI_01" width="400">

Als erstes geben wir ".schema" ein. Dadurch sehen wie alle Tabellen, welche sich in der Datenbankdatei befinden:
```
sqlite> .schema
CREATE TABLE todos (
        id INTEGER NOT NULL,
        title VARCHAR,
        description VARCHAR,
        priority INTEGER,
        complete BOOLEAN,
        PRIMARY KEY (id)
);
CREATE INDEX ix_todos_id ON todos (id);
```

Wir wollen nun einen Datensatz in unsere todos Tabelle einfügen. Dafür verwenden wir folgenden Befehl:
```
insert into todos (title, description, priority, complete) values ('Go to the store', 'Pick up eggs', 5, False);
```

Jetzt wählen wir alle Spalten aus und geben sie in der Konsole aus:
```
select * from todos;
```

Anschließend bekommen wir in der Konsole folgendes angezeigt:
```
1|Go to the store|Pick up eggs|5|0
```

Wir fügen noch ein todo, in unsere Tabelle hinzu:
```
insert into todos (title, description, priority, complete) values ('Cut the lawn', 'Grass is getting long', 3, False);
```

Jetzt wählen wir alle Spalten aus und geben sie in der Konsole aus:
```
select * from todos;
```

Anschließend bekommen wir in der Konsole folgendes angezeigt:
```
1|Go to the store|Pick up eggs|5|0
2|Cut the lawn|Grass is getting long|3|0
```

Wir erstelle noch einen weiteren Datensatz in unserer Tabelle:
```
insert into todos (title, description, priority, complete) values ('Feed the dog', 'He is getting hungry', 5, False);
```

Und wir betrachten ein weiteres mal unsere Tabelle:
```
select * from todos;
```

Dabei sehen wir:
```
1|Go to the store|Pick up eggs|5|0
2|Cut the lawn|Grass is getting long|3|0
3|Feed the dog|He is getting hungry|5|0
```

Falls man die Ausgabe in der Konsole nicht mag, kann man das Aussehen verändern, indem man den Modus ändert:
```
.mode column
```
Wenn wir jetzt alle Datenauslesen mit "select * from todos;", dann sieht die Ausgabe anders aus. Weitere mögliche Stiele sind:
- .mode markdown
- .mode box
- .mode table
- .mode list (Standard)

Wir erstellen ein weiteres Element:
```
insert into todos (title, description, priority, complete) values ('Test element', 'He is getting hungry', 5, False);
```

Wenn man den Modus Markdown verwendet (.mode markdown), dann sieht unsere Datenbank nun so aus:
```
| id |      title      |      description      | priority | complete |
|----|-----------------|-----------------------|----------|----------|
| 1  | Go to the store | Pick up eggs          | 5        | 0        |
| 2  | Cut the lawn    | Grass is getting long | 3        | 0        |
| 3  | Feed the dog    | He is getting hungry  | 5        | 0        |
| 4  | Test element    | He is getting hungry  | 5        | 0        |
```

Wir wollen nun dieses "Test element" löschen, dazu kann man folgendne Befehl verwenden:
```
delete from todos where id = 4;
```

Wir erstellen erneut ein neues Element:
```
insert into todos (title, description, priority, complete) values ('A new test element', 'He is getting hungry', 5, False);
```

Wir sehen dass die "id", erneut auf 4 gesetzt wurde, da es sich be ider Spalte "id", um einen sogenannten primary key handelt. Ich lösche jedoch dieses Element wieder:
```
delete from todos where id = 4;
```

Dies sollte eine kurze Wiederholung von einfachen SQL und Datenbanken sein. Jetzt wollen wir einen API-Endpunkt erstellen, mit demm wir alle Dateneinträge aus der Datenbank abrufen können. In der main.py Datei sorgt die Zeile:
```python
models.Base.metadata.create_all(bind=engine)
```
dafür, dass die Tabellen erstellt werden, wenn sie noch nicht existieren. Das Problem ist jedoch, dass wenn die Datenbank bereits existiert, dann werden bestehende Tabellen (z.B. todos) nicht automatisch aktualisiert. Wenn man also in der Datei models.y die "Todos" Klasse ändert, indem man z.B. eine neue Spalte einfügt, wird die bestehende Tabelle sich nicht ändern! Die einfachste Lösung für diese Problematik ist, einfach die Datenbankdatei erstmal zu löschen, anstelle sich um komplizierte Migrationslösungen gedancken zu machen, dies ist in der Entwicklungsphase häufig der Fall.
<br>
<br>
In der Produktion können wir natürlich nicht einfach so die Datenbankdatei löschen. Wir werden aber in Zukunft lernen, wie man dieses Problem durch ein Migrations-Tool umgeht. Dazu schauen wir un im Laufe des Kurses Alembic (Migrations-Tool) an.
<br>
<br>
Als erstes erstellen wir in der main.py Datei, eine sogenannte "Dependency Injection-Funktion". Diese Funktion verwendet FastAPI, um eine Datenbankverbindung zu öffnen und sicher zu schließen. Bei jeder Anfrage wird also eine eigene Datenbankverbidung geöffnet und am Ende der Anfrage automatisch geschlossen:

```python
from fastapi import FastAPI
import models
from database import engine, SessionLocal

app = FastAPI()

models.Base.metadata.create_all(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
```

- "db = SessionLocal()" erstellt eine neue SQLAlchemy-Session, die mit der SQLite-Datenbank verbunden ist.
- "yield db" sagt "Hier ist die Datenbank-Verbindung, nutze sie! Und wenn du fertig bist, schließe sie automatisch." Dadurch kann FastAPI die Verbindung während einer Anfrage nutzen und sie danach sauber schließen.
- "db.close()" Schließt die Datenbankverbindung, sobald die Anfrage abgeschlossen ist. Das verhindert Speicherlecks und hält die Anwendung effizient.

Nun ist es an der Zeit den API-Endpunkt zu erstellen. Dazu müssen wir einiges importieren:
- "Depends" stellt sicher, dass FastAPI eine bestimmte Funktion (z. B. get_db()) automatisch aufruft, wenn sie gebraucht wird.
- "Annotated" ist eine moderne Art, um zu sagen: „Dieser Parameter braucht eine bestimmte Funktion oder ein bestimmtes Verhalten.“
- "Session" ist das Werkzeug, mit dem FastAPI Daten aus der Datenbank abruft oder speichert.

So sieht erstmal der Endpunkt aus:

```python
from fastapi import FastAPI, Depends
import models
from database import engine, SessionLocal
from typing import Annotated
from sqlalchemy.orm import Session

app = FastAPI()

models.Base.metadata.create_all(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
        
@app.get("/")
async def read_all(db: Annotated[Session, Depends(get_db)]):
    pass
```

Die Variable "db" ist für die Datenbankverbindung zuständig, weswegen sie für die Abfragen genutzt wird. Dabei ist "Session der Typ der Variable, wodurch wir wissen, dass es sich bei "db" um eine Datenbanl-Session handelt. Durch "Depends(get_db)" ruft FastAPI automatisch die Funktion "get_db()" auf, um eine Datenbankverbindung zu erstellen. Dadurch müssen wir uns nicht manuell über das Öffnen und Schließen der Verbindung kümmern. "Annotated" ist einfach eine moderne Art, mehrere Informationen zu einer Variable zu geben.
<br>
<br>
Jetzt müssen wir in dem Funktions-Body die Logik implementieren, um alle Datenbankeinträge abzurufen. Vorher importieren wir noch aus "models" die "Todos" Klasse:

```python
from fastapi import FastAPI, Depends
import models
from models import Todos
from database import engine, SessionLocal
from typing import Annotated
from sqlalchemy.orm import Session

app = FastAPI()

models.Base.metadata.create_all(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
        
@app.get("/")
async def read_all(db: Annotated[Session, Depends(get_db)]):
    return db.query(Todos).all()
```

Dabei führt "db.query(Todos).all()" eine SQL-Abfrage aus, ohne direkt SQL zu schreiben. Diese Abfrage liest einfach alle Todos-Einträge aus der Datenbank. Durch "all()" werden alle gefundenen Einträge als Liste zurückgegeben. Wir wollen natürlich diesen Endpunkt testen und starten den uvicorn Server:
```
uvicorn main:app --reload
```

<img src="../img/FastAPI_66.png" alt="FastAPI_01" width="600">

Damit wir die Dependency Injection nicht jedes Mal neu schreiben müssen, wenn wir eine Datenbankverbindung erstellen möchten, lagern wir sie in die Variable db_dependency aus. Dadurch können wir einfach db_dependency als Parameter an die API-Endpunkte übergeben.
Das macht den Code kürzer, übersichtlicher und erleichtert spätere Änderungen, falls sich die Art der Datenbankverbindung ändern sollte:

```python
from fastapi import FastAPI, Depends
import models
from models import Todos
from database import engine, SessionLocal
from typing import Annotated
from sqlalchemy.orm import Session

app = FastAPI()

models.Base.metadata.create_all(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
        
db_dependency = Annotated[Session, Depends(get_db)]
        
@app.get("/")
async def read_all(db: db_dependency):
    return db.query(Todos).all()
```

Wir fügen einen neuen Endpunkt hinzu, um ein Todo nach seiner "id" zu filtern und auszugeben:

```python
from fastapi import FastAPI, Depends, HTTPException
import models
from models import Todos
from database import engine, SessionLocal
from typing import Annotated
from sqlalchemy.orm import Session

app = FastAPI()

models.Base.metadata.create_all(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
        
db_dependency = Annotated[Session, Depends(get_db)]
        
@app.get("/")
async def read_all(db: db_dependency):
    return db.query(Todos).all()

@app.get("/todo/{todo_id}")
async def read_todo(db: db_dependency, todo_id: int):
    todo_model = db.query(Todos).filter(Todos.id == todo_id).first()
    if todo_model is not None:
        return todo_model
    raise HTTPException(status_code=404, detail="Todo not found.")
```

- "HTTPException" wird benötigt, um eine Fehlermeldung zurückzugeben, falls das To-Do nicht gefunden wird.
- "db.query(Todos)" ruft alle To-Dos aus der Datenbank ab.
- ".filter(Todos.id == todo_id)" wählt nur das To-Do aus, das die gesuchte ID hat.
- ".first()" gibt das erste gefundene To-Do zurück oder None, falls nichts gefunden wurde.

Wir testen diesen Endpunkt, indem wir als "id" die "0" und die "1" probieren:

<img src="../img/FastAPI_67.png" alt="FastAPI_01" width="600">

<img src="../img/FastAPI_68.png" alt="FastAPI_01" width="600">

Wir erweitern den Endpunkt mit noch mehr Funktionalitäten:
- der Pfadparameter darf nicht kleiner "0" sein (Datenvalidierung)
- bei Erfolg soll der Statuscode "200 ok" zurückgegeben werden (ist bei FastAPI soweiso standardmäßig, jedoch sieht der Code sauberer aus)

```python
from fastapi import FastAPI, Depends, HTTPException, Path
import models
from models import Todos
from database import engine, SessionLocal
from typing import Annotated
from sqlalchemy.orm import Session
from starlette import status

app = FastAPI()

models.Base.metadata.create_all(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
        
db_dependency = Annotated[Session, Depends(get_db)]
        
@app.get("/")
async def read_all(db: db_dependency):
    return db.query(Todos).all()

@app.get("/todo/{todo_id}", status_code=status.HTTP_200_OK)
async def read_todo(db: db_dependency, todo_id: int = Path(gt=0)):
    todo_model = db.query(Todos).filter(Todos.id == todo_id).first()
    if todo_model is not None:
        return todo_model
    raise HTTPException(status_code=404, detail="Todo not found.")
```

Wir testen den Endppunkt mit nicht zulässigen Pfadparametern:

<img src="../img/FastAPI_69.png" alt="FastAPI_01" width="600">

Wir erweitern auch den Endpnkt "/", indem wir alle Datenbankeinträge auslesen:

```python
@app.get("/", status_code=status.HTTP_200_OK)
async def read_all(db: db_dependency):
    return db.query(Todos).all()
```

Jetzt wollen wir noch einen Endpunkt erstellen, mit dem wir ein neues Todo erstellen und in die Datenbank speichern. Als erstes erstellen wir ein Pydantic Datenmodell, das zur Validierung von Anfragedaten dient. Dazu wird aus dem Pydantic Modul die Klasse "BaseModel" benötigt. Das Datenmodell beichnen wir als "TodoRequests":

```python
from fastapi import FastAPI, Depends, HTTPException, Path
import models
from models import Todos
from database import engine, SessionLocal
from typing import Annotated
from sqlalchemy.orm import Session
from starlette import status
from pydantic import BaseModel, Field

app = FastAPI()

models.Base.metadata.create_all(bind=engine)

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
        
db_dependency = Annotated[Session, Depends(get_db)]

class TodoRequest(BaseModel):
    title: str = Field(min_length=3)
    description: str = Field(min_length=3, max_length=100)
    priority: int = Field(gt=0, lt=6)
    complete: bool
    
@app.get("/", status_code=status.HTTP_200_OK)
async def read_all(db: db_dependency):
    return db.query(Todos).all()

@app.get("/todo/{todo_id}", status_code=status.HTTP_200_OK)
async def read_todo(db: db_dependency, todo_id: int = Path(gt=0)):
    todo_model = db.query(Todos).filter(Todos.id == todo_id).first()
    if todo_model is not None:
        return todo_model
    raise HTTPException(status_code=404, detail="Todo not found.")
```

Jetzt können wir den neuen Endpunkt erstellen um über POST-Request ein neues Todo anzulegen:

```python
@app.post("/todo", status_code=status.HTTP_201_CREATED)
async def create_todo(db: db_dependency, todo_request: TodoRequest):
    todo_model = Todos(**todo_request.model_dump())
    db.add(todo_model)
    db.commit()
```

"todo_model = Todos(**todo_request.model_dump())":
- "todo_request" enthält die Nutzereingaben (Titel, Beschreibung, Priorität, etc.).
- "model_dump()" konvertiert das Pydantic-Objekt in ein Dictionary.
- "Todos(**todo_request.model_dump())" erstellt ein neues SQLAlchemy-Objekt mit diesen Werten.

"db.add(todo_model)":
- Fügt das neue To-Do zur Datenbank hinzu (aber speichert es noch nicht).

"db.commit()":
- Speichert die Änderungen dauerhaft in der Datenbank.

Wir testen nun den neuen Endpunkt mit dem folgenden Request-Body:
```
{
  "title": "Learn FastAPI",
  "description": "So I can learn how to create API Endpoints",
  "priority": 5,
  "complete": false
}
```

<img src="../img/FastAPI_70.png" alt="FastAPI_01" width="600">

Durch die GET-Anfrage, kann man alle Datenbankeinträge auslesen und prüfen ob der neue Eintrag vorhanden ist:

<img src="../img/FastAPI_71.png" alt="FastAPI_01" width="600">

Wir erweitern unsere API mit einem neuen Endpunkt, um die Datenbankeinträge zu aktualisieren:

```python
@app.put("/todo/{todo_id}", status_code=status.HTTP_204_NO_CONTENT)
async def update_todo(db: db_dependency, todo_id: int, todo_request: TodoRequest):
    todo_model = db.query(Todos).filter(Todos.id == todo_id).first()
    if todo_model is None:
        raise HTTPException(status_code=404, detail="Todo not found.")
    
    todo_model.title = todo_request.title
    todo_model.description = todo_request.description
    todo_model.priority = todo_request.priority
    todo_model.complete = todo_request.complete
    
    db.add(todo_model)
    db.commit()
```

- "todo_request: TodoRequest" enthält die neuen Daten, die an das bestehende To-Do übergeben werden.
- Mit "db.query(Todos).filter(Todos.id == todo_id).first()" suchen wir nach dem To-Do mit der todo_id.
- Das "todo_model" wird mit den neuen Daten aus "todo_request" überschrieben.
- "db.add(todo_model)" markiert das todo_model als aktualisiert.
- "db.commit()" speichert die Änderungen dauerhaft in der Datenbank.

Nun testen wir den neuen Endpunkt mit den folgenden Request-Body:
```
{
  "title": "Test Title",
  "description": "Test Description",
  "priority": 2,
  "complete": false
}
```

<img src="../img/FastAPI_72.png" alt="FastAPI_01" width="600">

Anschließend lesen wir alle Datenbankeinträge aus und sehen unseren aktualisierten Eintrag:

<img src="../img/FastAPI_73.png" alt="FastAPI_01" width="600">

Wir fügen zu dem Endpunkt noch eine Pathvalidation hinzu, damit die "todo_id" keine Werte kleiner gleich Null akzeptiert:

```python
@app.put("/todo/{todo_id}", status_code=status.HTTP_204_NO_CONTENT)
async def update_todo(db: db_dependency, todo_id: int = Path(gt=0), todo_request: TodoRequest):
    todo_model = db.query(Todos).filter(Todos.id == todo_id).first()
    if todo_model is None:
        raise HTTPException(status_code=404, detail="Todo not found.")
    
    todo_model.title = todo_request.title
    todo_model.description = todo_request.description
    todo_model.priority = todo_request.priority
    todo_model.complete = todo_request.complete
    
    db.add(todo_model)
    db.commit()
```

Jedoch bekommen wir einen Fehler:
```
Non-default argument follows default argumentPylance
(parameter) todo_request: TodoRequest
```

Wir müssen den Parameter "todo_request: TodoRequest" vor allem definiert werden, was mit Pfadvalidierung zu tun hat. Deswegen passen wir die Parameterreihenfolge an:

```python
@app.put("/todo/{todo_id}", status_code=status.HTTP_204_NO_CONTENT)
async def update_todo(db: db_dependency, todo_request: TodoRequest, todo_id: int = Path(gt=0)):
    todo_model = db.query(Todos).filter(Todos.id == todo_id).first()
    if todo_model is None:
        raise HTTPException(status_code=404, detail="Todo not found.")
    
    todo_model.title = todo_request.title
    todo_model.description = todo_request.description
    todo_model.priority = todo_request.priority
    todo_model.complete = todo_request.complete
    
    db.add(todo_model)
    db.commit()
```

Wenn wir nun diesen Endpunkt mit der id "-1" testen, bekommen wir einen Validation Error:

<img src="../img/FastAPI_74.png" alt="FastAPI_01" width="600">

Jetzt fügen wir einen Endpunkt hinzu, um bestimmte Datenbankeinträge zu löschen:

```python
@app.delete("/todo/{todo_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_todo(db: db_dependency, todo_id: int = Path(gt=0)):
    todo_model = db.query(Todos).filter(Todos.id == todo_id).first()
    if todo_model is None:
        raise HTTPException(status_code=404, detail="Todo not found.")
    db.query(Todos).filter(Todos.id == todo_id).delete()
    db.commit()
```

Wir testen diesen Endpunkt, indem wir einen Datenbankeintrag mit der "id" von "1" löschen:

<img src="../img/FastAPI_75.png" alt="FastAPI_01" width="600">

Wenn wir alle Todos abrufen, so ist das Todo mit der "id=1" nicht vorhanden:

<img src="../img/FastAPI_76.png" alt="FastAPI_01" width="600">